## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
# city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Nishihara,26.1842,127.7558,68.00,63,40,9.22,JP,scattered clouds
1,1,Bluff,-46.6000,168.3333,50.00,84,66,1.99,NZ,broken clouds
2,2,Del Rio,29.3627,-100.8968,60.80,48,90,8.05,US,overcast clouds
3,3,Kaukauna,44.2780,-88.2720,39.00,52,90,17.27,US,overcast clouds
4,4,Krasnyy Chikoy,50.3643,108.7512,43.32,45,8,4.63,RU,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Nishihara,26.1842,127.7558,68.00,63,40,9.22,JP,scattered clouds
1,1,Bluff,-46.6000,168.3333,50.00,84,66,1.99,NZ,broken clouds
2,2,Del Rio,29.3627,-100.8968,60.80,48,90,8.05,US,overcast clouds
5,5,Ridgecrest,35.6225,-117.6709,62.01,29,1,3.65,US,clear sky
6,6,Mayo,38.8876,-76.5119,57.00,76,90,5.75,US,overcast clouds
7,7,Hermanus,-34.4187,19.2345,66.00,63,100,11.99,ZA,overcast clouds
8,8,Faanui,-16.4833,-151.7500,81.82,71,10,13.04,PF,clear sky
12,12,Port Elizabeth,-33.9180,25.5701,60.80,82,90,8.05,ZA,light rain
13,13,Albany,42.6001,-73.9662,57.99,46,40,2.62,US,scattered clouds
17,17,Chuy,-33.6971,-53.4616,71.06,58,77,10.56,UY,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                55
City                   55
Lat                    55
Lng                    55
Max Temp               55
Humidity               55
Cloudiness             55
Wind Speed             55
Country                55
Current Description    55
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nishihara,JP,68.00,scattered clouds,26.1842,127.7558,
1,Bluff,NZ,50.00,broken clouds,-46.6000,168.3333,
2,Del Rio,US,60.80,overcast clouds,29.3627,-100.8968,
5,Ridgecrest,US,62.01,clear sky,35.6225,-117.6709,
6,Mayo,US,57.00,overcast clouds,38.8876,-76.5119,
7,Hermanus,ZA,66.00,overcast clouds,-34.4187,19.2345,
8,Faanui,PF,81.82,clear sky,-16.4833,-151.7500,
12,Port Elizabeth,ZA,60.80,light rain,-33.9180,25.5701,
13,Albany,US,57.99,scattered clouds,42.6001,-73.9662,
17,Chuy,UY,71.06,broken clouds,-33.6971,-53.4616,


In [9]:
hotel_df["Lat"]

1    -33.0153
11     1.5500
17    25.2867
23   -16.4833
25    19.7297
31   -33.5906
32    24.5557
35   -22.9661
36     9.1104
38    -5.1989
40   -21.0096
48    11.6636
49   -21.2078
50     5.6592
54    25.8523
56     5.4112
63   -23.1203
65    20.8833
67     0.1333
69   -13.8833
72    22.0752
79   -20.7377
Name: Lat, dtype: float64

In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key, 
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [14]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nishihara,JP,68.00,scattered clouds,26.1842,127.7558,Minshuku Agaihama
1,Bluff,NZ,50.00,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
2,Del Rio,US,60.80,overcast clouds,29.3627,-100.8968,Holiday Inn Express & Suites del Rio
5,Ridgecrest,US,62.01,clear sky,35.6225,-117.6709,SpringHill Suites by Marriott Ridgecrest
6,Mayo,US,57.00,overcast clouds,38.8876,-76.5119,Resorts Resource Group International
7,Hermanus,ZA,66.00,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
8,Faanui,PF,81.82,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
12,Port Elizabeth,ZA,60.80,light rain,-33.9180,25.5701,39 On Nile Guest House
13,Albany,US,57.99,scattered clouds,42.6001,-73.9662,
17,Chuy,UY,71.06,broken clouds,-33.6971,-53.4616,Nuevo Hotel Plaza


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# <dt>Description</dt><dd>{Current Description}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """
info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>

# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))